In [1]:
import pandas
import numpy
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
"""
This tutorial shows how to generate adversarial examples
using FGSM in black-box setting.
The original paper can be found at:
https://arxiv.org/abs/1602.02697
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np
from six.moves import xrange

import logging
import tensorflow as tf
from tensorflow.python.platform import flags

from cleverhans.utils_mnist import data_mnist
from cleverhans.utils import to_categorical
from cleverhans.utils import set_log_level
from cleverhans.utils_tf import model_train, model_eval, batch_eval
from cleverhans.attacks import FastGradientMethod
from cleverhans.attacks_tf import jacobian_graph, jacobian_augmentation

from cleverhans_tutorials.tutorial_models import make_basic_cnn, MLP
from cleverhans_tutorials.tutorial_models import Flatten, Linear, ReLU, Softmax
from cleverhans.utils import TemporaryLogLevel

from lad import lad
from scipy.spatial.distance import euclidean

FLAGS = flags.FLAGS

# Functions

## Data

In [2]:
'''
MOONS
'''
def get_moon():
    X, y = make_moons(noise=0.3, random_state=1, n_samples=10000)
    y2 = np.zeros((X.shape[0],2))
    for k in range(len(y)):
        y2[k][y[k]] = 1
    return X, y2
DATASETS_ = {'moons':get_moon}

## Training a black-box

In [3]:
'''
PAPERNOT BB
'''
def Papernot_bbox(sess, x, y, X_train, Y_train, X_test, Y_test,
              nb_epochs, batch_size, learning_rate,
              rng):
    """
    Define and train a model that simulates the "remote"
    black-box oracle described in the original paper.
    :param sess: the TF session
    :param x: the input placeholder for MNIST
    :param y: the ouput placeholder for MNIST
    :param X_train: the training data for the oracle
    :param Y_train: the training labels for the oracle
    :param X_test: the testing data for the oracle
    :param Y_test: the testing labels for the oracle
    :param nb_epochs: number of epochs to train model
    :param batch_size: size of training batches
    :param learning_rate: learning rate for training
    :param rng: numpy.random.RandomState
    :return:
    """

    # Define TF model graph (for the black-box model)
    model = make_basic_cnn()
    predictions = model(x)
    print("Defined TensorFlow model graph.")

    # Train an MNIST model
    train_params = {
        'nb_epochs': nb_epochs,
        'batch_size': batch_size,
        'learning_rate': learning_rate
    }
    model_train(sess, x, y, predictions, X_train, Y_train,
                args=train_params, rng=rng)

    # Print out the accuracy on legitimate data
    eval_params = {'batch_size': batch_size}
    accuracy = model_eval(sess, x, y, predictions, X_test, Y_test,
                          args=eval_params)
    print('Test accuracy of black-box on legitimate test '
          'examples: ' + str(accuracy))

    return model, predictions, accuracy

def RF_bbox(X_train, Y_train, X_test, Y_test):
    # Define RF model graph (for the black-box model)

    model = RandomForestClassifier(n_estimators=100, n_jobs=-1).fit(X_train, Y_train)
    
    # Print out the accuracy on legitimate data
    #predictions = model.predict_proba(X_test)[1] TEST CHANGER PREDICTIONS > FONCTION
    predictions=lambda x: model.predict_proba(x)[1] #predict_proba required ou alors changer du code (argmax et compagnie) de papernot
    
    accuracy = accuracy_score(Y_test, model.predict(X_test))
    #roc_auc = roc_auc_score(Y_test, predictions[1][:,1])
    print('Test accuracy of black-box on legitimate test '
          'examples: ' + str(accuracy))
    #print('Test ROC AUC of black-box on legitimate test ' 'examples: ' + str(roc_auc))
        
    
    return model, predictions, accuracy
    
BB_MODELS_ = {'dnn': Papernot_bbox,
            'rf': RF_bbox}
#ne pas utiliser dnn ca marche pas pour le moment

## Papernot Surrogate

In [4]:
def setup_tutorial():
    """
    Helper function to check correct configuration of tf for tutorial
    :return: True if setup checks completed
    """

    # Set TF random seed to improve reproducibility
    tf.set_random_seed(1234)

    return True
def substitute_model(img_rows=1, img_cols=2, nb_classes=2):
    """
    Defines the model architecture to be used by the substitute. Use
    the example model interface.
    :param img_rows: number of rows in input
    :param img_cols: number of columns in input
    :param nb_classes: number of classes in output
    :return: tensorflow model
    """
    input_shape = (None, img_rows, img_cols, 1) #on garde format d'origine parce qu'on comprend pas grand chose mais on change valeurs

    # Define a fully connected model (it's different than the black-box)
    '''layers = [Flatten(),
              Linear(200),
              ReLU(),
              Linear(200),
              ReLU(),
              Linear(nb_classes),
              Softmax()]'''
    layers = [Flatten(), Linear(nb_classes), Softmax()] #surrogate simplifié

    return MLP(layers, input_shape)


def train_sub(sess, x, y, bbox_preds, X_sub, Y_sub, nb_classes,
              nb_epochs_s, batch_size, learning_rate, data_aug, lmbda,
              rng):
    """
    This function creates the substitute by alternatively
    augmenting the training data and training the substitute.
    :param sess: TF session
    :param x: input TF placeholder
    :param y: output TF placeholder
    :param bbox_preds: output of black-box model predictions
    :param X_sub: initial substitute training data
    :param Y_sub: initial substitute training labels
    :param nb_classes: number of output classes
    :param nb_epochs_s: number of epochs to train substitute model
    :param batch_size: size of training batches
    :param learning_rate: learning rate for training
    :param data_aug: number of times substitute training data is augmented
    :param lmbda: lambda from arxiv.org/abs/1602.02697
    :param rng: numpy.random.RandomState instance
    :return:
    """
    # Define TF model graph (for the black-box model)
    model_sub = substitute_model()
    preds_sub = model_sub(x)
    print("Defined TensorFlow model graph for the substitute.")

    # Define the Jacobian symbolically using TensorFlow
    grads = jacobian_graph(preds_sub, x, nb_classes)
    # Train the substitute and augment dataset alternatively
    for rho in xrange(data_aug):
        print("Substitute training epoch #" + str(rho))
        train_params = {
            'nb_epochs': nb_epochs_s,
            'batch_size': batch_size,
            'learning_rate': learning_rate
        }
        with TemporaryLogLevel(logging.WARNING, "cleverhans.utils.tf"):
            model_train(sess, x, y, preds_sub, X_sub,
                        to_categorical(Y_sub, nb_classes),
                        init_all=False, args=train_params, rng=rng)

        # If we are not at last substitute training iteration, augment dataset
        if rho < data_aug - 1:
            print("Augmenting substitute training data.")
            # Perform the Jacobian augmentation
            lmbda_coef = 2 * int(int(rho / 3) != 0) - 1
            print('a')
            X_sub = jacobian_augmentation(sess, x, X_sub, Y_sub, grads,
                                          lmbda_coef * lmbda)
            print('b')
            
            print("Labeling substitute training data.")
            # Label the newly generated synthetic points using the black-box
            Y_sub = np.hstack([Y_sub, Y_sub])
            X_sub_prev = X_sub[int(len(X_sub)/2):] #on a double le dataset donc prev = ce qu'il y a de nouveau = la moitie
            eval_params = {'batch_size': batch_size}
            
            #bbox_preds = tf.convert_to_tensor(bbox_preds, dtype=tf.float32) TEST CHANGER PREDICTIONS > FONCTION           
            #bbox_val = batch_eval2(sess, [x], [bbox_preds], [X_sub_prev], args=eval_params)[0] TEST CHANGER PREDICTIONS > FONCTION
            bbox_val = bbox_preds(X_sub_prev) #normalement batch eval sert juste à sortir les preds...?
            # Note here that we take the argmax because the adversary
            # only has access to the label (not the probabilities) output
            # by the black-box model
            Y_sub[int(len(X_sub)/2):] = np.argmax(bbox_val, axis=1)
    return model_sub, preds_sub 

In [5]:

'''
PAS UTILISE DANS LA VERSION ACTUELLE
MAIS LAISSER AU CAS OU SVP
'''


def batch_eval2(sess, tf_inputs, tf_outputs, numpy_inputs, feed=None,
               args=None):
    #print('tf outputs', tf_outputs[0].shape) TEST CHANGER PREDICTIONS > FONCTION
    """
    A helper function that computes a tensor on numpy inputs by batches.
    :param sess:
    :param tf_inputs:
    :param tf_outputs:
    :param numpy_inputs:
    :param feed: An optional dictionary that is appended to the feeding
             dictionary before the session runs. Can be used to feed
             the learning phase of a Keras model for instance.
    :param args: dict or argparse `Namespace` object.
                 Should contain `batch_size`
    """
    args = _ArgsWrapper(args or {})

    assert args.batch_size, "Batch size was not given in args dict"

    n = len(numpy_inputs)
    assert n > 0
    assert n == len(tf_inputs)
    m = numpy_inputs[0].shape[0]
    for i in xrange(1, n):
        assert numpy_inputs[i].shape[0] == m
    out = []
    for _ in tf_outputs:
        out.append([])
    with sess.as_default():
        for start in xrange(0, m, args.batch_size):
            batch = start // args.batch_size
            if batch % 100 == 0 and batch > 0:
                _logger.debug("Batch " + str(batch))

            # Compute batch start and end indices
            start = batch * args.batch_size            
            numpy_input_batches = [numpy_input[start:end]
                                   for numpy_input in numpy_inputs]
            cur_batch_size = numpy_input_batches[0].shape[0]
            assert cur_batch_size <= args.batch_size
            
            for e in numpy_input_batches:
                assert e.shape[0] == cur_batch_size

            feed_dict = dict(zip(tf_inputs, numpy_input_batches))
            
            
            #print('feed_dict', feed_dict)
            print("feed",feed)
            if feed is not None:
                feed_dict.update(feed)
            #numpy_output_batches = sess.run(tf_outputs, feed_dict=feed_dict) #PROBLEME ICI : AU LIEU DES BATCHES, TOUT LE DATASET DE PRED
            #ici cest retourner les predictions des batchs.
            # donc liste de pred(batchs)
            for batch in numpy_input_batches: #TEST CHANGER PREDICTIONS > FONCTION
                numpy_output_batches.append(tf_outputs(batch))
            print("len np output bitches", len(numpy_output_batches))
            for e in numpy_output_batches:
                print("e", e)
                print("e shape", e.shape)
                print(" cur batch size useless", cur_batch_size)
                assert e.shape[0] == cur_batch_size, e.shape #ERREUR ICI
            for out_elem, numpy_output_batch in zip(out, numpy_output_batches):
                out_elem.append(numpy_output_batch)

    out = [np.concatenate(x, axis=0) for x in out]
    for e in out:
        assert e.shape[0] == m, e.shape
    return out




class _ArgsWrapper(object):

    """
    Wrapper that allows attribute access to dictionaries
    """

    def __init__(self, args):
        if not isinstance(args, dict):
            args = vars(args)
        self.args = args

    def __getattr__(self, name):
        return self.args.get(name)

Usage: 
print("Training the substitute model.")
    train_sub_out = train_sub(sess, x, y, bbox_preds, X_sub, Y_sub,
                              nb_classes, nb_epochs_s, batch_size,
                              learning_rate, data_aug, lmbda, rng=rng)
    model_sub, preds_sub = train_sub_out

# Our surrogate

# Local Fidelity

In [6]:
def get_random_points_hypersphere(x_center, radius_, n_points_):

        res = []
        while len(res) < n_points_:
        
            n_points_left_ = n_points_ - len(res)
            # About half the points are lost in the test hypercube => hypersphere
            lbound = numpy.repeat([x_center.values-(radius_/2.)], n_points_left_*2, axis=0)
            hbound = numpy.repeat([x_center.values+(radius_/2.)], n_points_left_*2, axis=0)
            points = numpy.random.uniform(low=lbound, high=hbound)
            # Check if x_generated is within hypersphere (if kind=='hypersphere')
            for x_generated in points:
                if euclidean(x_generated, x_center.values) < radius_:
                    res.append(x_generated)
                if len(res) == n_points_:
                    break

        return pandas.DataFrame(numpy.array(res))

# Framework

In [8]:

def main_fidelity():
    accuracies = {}
    fidelities = {}
    
    
    # Seed random number generator so tutorial is reproducible
    rng = np.random.RandomState([2017, 8, 30])

    # Thibault: Tensorflow stuff
    set_log_level(logging.DEBUG)
    assert setup_tutorial()
    sess = tf.Session()
    
    
    
    # Data
    X, Y = DATASETS_['moons']()
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)
    X_sub = X_test[:holdout]
    Y_sub = np.argmax(Y_test[:holdout], axis=1)

    ## Redefine test set as remaining samples unavailable to adversaries
    ### N.B Thibault: c'est pour le substitute de Papernot
    X_test = X_test[holdout:]
    Y_test = Y_test[holdout:]
    print("Training black box on",X_train.shape[0], "examples")
    print('Testing black box and substitute on', X_test.shape[0],' examples')
    print("Using ", holdout, " examples to start PP substitute")
    ## Define input and output TF placeholders
    ### N.B. Thibault: restes de Tensorflow, utilisé pour le substitute de Papernot...
    x = tf.placeholder(tf.float32, shape=(None, 2))
    y = tf.placeholder(tf.float32, shape=(None, 2))  
    
    
    
    # Instance to explain
    x_toexplain = pandas.Series(X_test[0]).copy()
    support_x_ = np.array(get_random_points_hypersphere(x_toexplain, radius_=radius, n_points_=1000))
    
    
    
    # Simulate the black-box model
    print("Preparing the black-box model.")
    prep_bbox_out = BB_MODELS_['rf'](X_train, Y_train, X_test, Y_test)
    bb_model, bbox_preds, accuracies['bbox'] = prep_bbox_out #bbox_preds fonction predict
    
    # Train PAPERNOT substitute
    print("Training the Pépèrenot substitute model.")
    train_sub_pap = train_sub(sess, x, y, bbox_preds, X_sub, Y_sub,
                              nb_classes, nb_epochs_s, batch_size,
                              learning_rate, data_aug, lmbda, rng=rng)
    model_sub, preds_sub = train_sub_pap
    eval_params = {'batch_size': batch_size}
    pap_acc = model_eval(sess, x, y, preds_sub, X_test, Y_test, args=eval_params) 
    pap_fid = model_eval(sess, x, y, preds_sub, support_x_, bbox_preds(support_x_) , args=eval_params)
    accuracies['papernot'] = pap_acc
    fidelities['papernot'] = pap_fid
    
    
    # Train OUR subtitute
    print("Training Local Surrogate substitute model.")
    pred = bb_model.predict
    bb_model.predict = lambda x: pred(x)[:,1]
    _, train_sub_ls = lad.LocalSurrogate(pandas.DataFrame(X), blackbox=bb_model, n_support_points=100, max_depth=3).get_local_surrogate(x_toexplain)
    #ls_acc = accuracy_score(train_sub_ls.predict(X_test), Y_test)
    ls_fid = accuracy_score(train_sub_ls.predict(support_x_), bb_model.predict(support_x_))
    #accuracies['localsurrogate'] = ls_acc
    fidelities['localsurrogate'] = ls_fid
    

    
    return fidelities, accuracies



nb_classes=2 #
batch_size=120 #
learning_rate=0.001 #
nb_epochs=1 # Nombre d'itération bbox osef
holdout=150 # Nombre d'exemples utilisés au début pour générer data (Pap-substitute)
data_aug=3 # Nombre d'itérations d'augmentation du dataset {IMPORTANT pour Pap-substitute}
nb_epochs_s=1 # Nombre d'itérations pour train substitute
lmbda=0.1 # params exploration pour augmentation data
radius = 0.5 # NEW


main_fidelity()

Training black box on 8000 examples
Testing black box and substitute on 1850  examples
Using  150  examples to start PP substitute
Preparing the black-box model.
Test accuracy of black-box on legitimate test examples: 0.9059459459459459
Training the Pépèrenot substitute model.
Defined TensorFlow model graph for the substitute.
Substitute training epoch #0


[INFO 2018-06-26 15:34:57,914 cleverhans] Epoch 0 took 0.019896507263183594 seconds
[INFO 2018-06-26 15:34:57,915 cleverhans] Completed model training.


Augmenting substitute training data.
a
b
Labeling substitute training data.
Substitute training epoch #1


[INFO 2018-06-26 15:35:01,660 cleverhans] Epoch 0 took 0.0197751522064209 seconds
[INFO 2018-06-26 15:35:01,661 cleverhans] Completed model training.


Augmenting substitute training data.
a
b
Labeling substitute training data.
Substitute training epoch #2


[INFO 2018-06-26 15:35:11,152 cleverhans] Epoch 0 took 0.022075414657592773 seconds
[INFO 2018-06-26 15:35:11,153 cleverhans] Completed model training.


Training Local Surrogate substitute model.


({'localsurrogate': 1.0, 'papernot': 0.0},
 {'bbox': 0.9059459459459459, 'papernot': 0.21027027027027026})

In [ ]:
pandas.Series(X[0])[0]

In [ ]:
X, Y = DATASETS_['moons']()
X.min(axis=0), X[:,1].min()


support_points = []

while len(support_points) < 10:
    print(len(support_points))
    candidate_ = numpy.random.uniform(low=X.min(axis=0),
                               high=X.max(axis=0))
    if self.blackbox.predict([candidate_]).all() != self.blackbox.predict([x_toexplain]).all():
        support_points.append(candidate_)

support_points = pandas.DataFrame(support_points, columns=x_toexplain.index)